## 식당 검색어를 통해 네이버 식당 페이지 url 얻기

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import NoSuchElementException 

import time
import re
from bs4 import BeautifulSoup 
from tqdm import tqdm 


In [2]:
# 음식점 리스트 파일 불러들이기

file_name = './url검색용_분할자료/chunk_1500.csv'
df = pd.read_csv(f'./{file_name}')
df

,사업장명,업태구분명,소재지전화,소재지전체주소,도로명전체주소,좌표정보(x),좌표정보(y),영업상태명,naver_map_url,구,동,검색어,검색어2
0,유가네닭갈비개금점(1F),기타,051-891-3311,부산광역시 부산진구 가야동 632-5번지 (1층),"부산광역시 부산진구 가야대로498번길 28-3, 1층 (가야동)",384584.007571,185487.929276,영업/정상,NaN,부산진구,가야동,가야동 유가네닭갈비개금점(1F),가야동 유가네닭갈비개금점(1F) 051-891-3311
1,부흥,기타,NaN,부산광역시 부산진구 부전동 168-24 지상1층,"부산광역시 부산진구 서전로10번길 13, 지상1층 (부전동)",387692.080955,186146.055615,영업/정상,NaN,부산진구,부전동,부전동 부흥,부전동 부흥
2,처마집,기타,NaN,부산광역시 부산진구 부전동 168-45 1층,"부산광역시 부산진구 동천로85번길 40, 1층 (부전동)",387671.721134,186123.027658,영업/정상,NaN,부산진구,부전동,부전동 처마집,부전동 처마집
3,승승(초읍점),한식,051-816-4491,부산광역시 부산진구 초읍동 269-14번지 지상1층일부,"부산광역시 부산진구 새싹로 202 (초읍동, 지상1층일부)",386612.328012,188034.194575,영업/정상,NaN,부산진구,초읍동,초읍동 승승(초읍점),초읍동 승승(초읍점) 051-816-4491
4,소나무집,한식,051-809-6650,부산광역시 부산진구 초읍동 463번지 지상1층일부),"부산광역시 부산진구 월드컵대로 465 (초읍동, 지상1층일부)",386685.176132,188925.522405,영업/정상,NaN,부산진구,초읍동,초읍동 소나무집,초읍동 소나무집 051-809-6650
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,마카오,한식,NaN,부산광역시 부산진구 부전동 240-6,"부산광역시 부산진구 신천대로62번길 67, 1층 (부전동)",387483.804616,185785.048132,영업/정상,NaN,부산진구,부전동,부전동 마카오,부전동 마카오
496,파커(Parker),기타,NaN,부산광역시 부산진구 전포동 308-3 스카이빌,"부산광역시 부산진구 전포대로216번길 26, 스카이빌 1층 (전포동)",388271.837860,186016.108666,영업/정상,NaN,부산진구,전포동,전포동 파커(Parker),전포동 파커(Parker)
497,광화문명가,한식,NaN,부산광역시 부산진구 부전동 256-6,"부산광역시 부산진구 서면로 56, 1층 타32호 (부전동)",387468.480136,185990.346087,영업/정상,NaN,부산진구,부전동,부전동 광화문명가,부전동 광화문명가
498,우리집밥상,한식,051-4663567,부산광역시 부산진구 부전동 475-12,"부산광역시 부산진구 부전로96번길 36, 1층 (부전동)",387327.064275,186320.465355,영업/정상,NaN,부산진구,부전동,부전동 우리집밥상,부전동 우리집밥상 051-4663567


In [3]:
driver = webdriver.Chrome(executable_path='C:/Users/ysj_1/Documents/ds_study/driver/chromedriver.exe') 
# 웹드라이버가 설치된 경로를 지정해주시면 됩니다.

C:\Users\ysj_1\AppData\Local\Temp\ipykernel_29136\1903894952.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='C:/Users/ysj_1/Documents/ds_study/driver/chromedriver.exe')


In [4]:

# 데이터프레임의 '검색어' 열에서 키워드를 하나씩 가져온다.
for i, keyword in tqdm(enumerate(df['검색어'].tolist())):

    # 현재 몇 번째 행의 키워드를 찾고 있는지 출력한다.
    print("이번에 찾을 키워드 :", i+1, f"/ {df.shape[0]} 행", keyword)

    try:  # 예외처리 시작
        # 검색 url 만들기
        naver_map_search_url = f'https://map.naver.com/v5/search/{keyword}/place'
        driver.get(naver_map_search_url)  # 검색 url 접속 = 검색하기
        time.sleep(4)  # 중요

        cu = driver.current_url  # 검색이 성공된 플레이스에 대한 개별 페이지
        res_code = re.findall(r"place/(\d+)", cu)  # 정규표현식으로 플레이스 코드 추출
        final_url = 'https://pcmap.place.naver.com/restaurant/' + \
            res_code[0]+'/review/visitor#'  # 리뷰 페이지 url 만들기

        print(final_url)  # 리뷰 페이지 url 출력
        # 데이터프레임의 'naver_map_url' 열에 리뷰 페이지 url 저장
        df['naver_map_url'][i] = final_url

    except IndexError:  # 만약 플레이스 코드를 찾을 수 없다면
        df['naver_map_url'][i] = ''  # 데이터프레임의 'naver_map_url' 열에 빈 문자열 저장
        print('none')  # 'none' 출력

    # 데이터프레임을 csv 파일로 저장한다.
    df.to_csv(f'./url_completed_to_{file_name}.csv', encoding='utf-8-sig')


432it [31:56,  4.26s/it]

https://pcmap.place.naver.com/restaurant/1234548205/review/visitor#
이번에 찾을 키워드 : 433 / 500 행 부전동 숯꾸닭


433it [32:00,  4.26s/it]

none
이번에 찾을 키워드 : 434 / 500 행 전포동 포도와인


434it [32:04,  4.24s/it]

...

https://pcmap.place.naver.com/restaurant/1714699730/review/visitor#
이번에 찾을 키워드 : 499 / 500 행 부전동 우리집밥상


499it [36:42,  4.38s/it]

https://pcmap.place.naver.com/restaurant/1972634868/review/visitor#
이번에 찾을 키워드 : 500 / 500 행 전포동 그릭하다


500it [36:46,  4.41s/it]

none


In [5]:
driver.quit()